In [1]:
import requests
import pandas as pd
import io
import sqlite3

# You must request an NSRDB api key from the link above
api_key = 'pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe'
# Your full name, use '+' instead of spaces.
name = 'Yuji'
# Your affiliation
affiliation = 'LeWagon'
# Your email address
email = 'yuji.ando16@gmail.com'
#year = [1999,2000,2001,2002,2003,2004,2005,2006,2007] # Kaoru
#year = [2008,2009,2010,2011,2012] # Afif
years = [2013,2014,2015,2016,2017] # Yuji

def getTask(years = None):
    conn = sqlite3.connect('download_tracker.sql')
    query = 'SELECT YEAR, POINT FROM files WHERE DONE = 0 limit 1'
    if years is not None:
        if type(years) == int:
            years = [years]
        yearstr = ','.join([str(y) for y in years])
        query = f'SELECT YEAR, POINT FROM files WHERE year IN ({yearstr}) AND DONE = 0 limit 1'
    task = pd.read_sql_query(query,conn)
    return task

def writeTask(year, point):
    conn = sqlite3.connect('download_tracker.sql')
    c = conn.cursor()
    query = f'UPDATE FILES SET DONE = 1 WHERE YEAR = {year} AND POINT = {point}'
    c.execute(query)
    conn.commit()
    conn.close()
    print(f'Completed record y:{year}, p:{point}')

def get_ghi_data(lat, lon, year, api_key,i):

    url = f'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv'
    params = {
        'api_key': api_key,
        'wkt': f"POINT({lon} {lat})",
        'names': year,
        'interval': '60',
        'full_name': 'Your Name',
        'email': email,
        'affiliation': affiliation,
        'reason': 'testing',
        'attributes': 'ghi,dni,dhi,clearsky_ghi,clearsky_dni,clearsky_dhi,solar_zenith_angle,air_temperature'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text), skiprows=2)
        df.to_csv(f'{year}_{i}_radiance.csv')
        return df
    else:
        print(f"Error fetching data for {lat}, {lon}: {response.status_code}")
        return None

In [2]:
import glob

conn = sqlite3.connect('download_tracker.sql')

task = getTask(years)
while task.shape[0]>0:
    year, pid = getTask(years).to_numpy()[0]
    LAT, LON =  pd.read_sql_query(f'SELECT Latitude, longitude FROM points WHERE I = {pid}',conn).to_numpy()[0]
    print(LAT, LON, year, api_key,pid)
    get_ghi_data(LAT, LON, year, api_key,pid)
    #check if the file is here
    chk = glob.glob(f'./{year}_{pid}_radiance.csv')
    if len(chk) > 0:
        print('file check Confirmed',chk[0])
        writeTask(year, pid)
    else:
        print('file check Failed')
        break
    #Get new task
    task = getTask(years)

41.27845761168134 -120.6932812839476 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1667
file check Confirmed ./2015_1667_radiance.csv
Completed record y:2015, p:1667
35.97394343993562 -117.82612274704778 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1668
file check Confirmed ./2015_1668_radiance.csv
Completed record y:2015, p:1668
35.112238127780344 -116.16731735908182 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1669
file check Confirmed ./2015_1669_radiance.csv
Completed record y:2015, p:1669
34.698019358330896 -119.76399019211044 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1670
file check Confirmed ./2015_1670_radiance.csv
Completed record y:2015, p:1670
36.953010444030305 -121.30334537027342 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1671
file check Confirmed ./2015_1671_radiance.csv
Completed record y:2015, p:1671
35.59111792557963 -118.51426005156942 2015 pQSWoAQlqkJxqIgPndzSze61FPxDbjhwCcmtKpNe 1672
file check Confirmed ./2015_1672_radiance.csv
Completed record y:2015, p:1

In [1]:
def percentComplete():
    conn = sqlite3.connect('download_tracker.sql')
    done = pd.read_sql_query(f'SELECT count(*) from files where done = 1',conn).to_numpy()[0][0]
    mada = pd.read_sql_query(f'SELECT count(*) from files where done = 0',conn).to_numpy()[0][0]
    print(done/(done+mada)*100,'percent completed')

percentComplete()

NameError: name 'sqlite3' is not defined